In [1]:
import pandas as pd
import torch
import matplotlib as mpl
import matplotlib.pyplot as plt
from datasets import DatasetDict, load_from_disk, Dataset, concatenate_datasets
import re

/opt/conda/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import json
with open('/opt/ml/input/data/ai_hub/ko_nia_clue0529_squad_all.json','r') as f:
    clue0529 = json.load(f)
with open('/opt/ml/input/data/ai_hub/ko_nia_noanswer_squad_all.json','r') as f:
    noanswer = json.load(f)
with open('/opt/ml/input/data/ai_hub/ko_nia_normal_squad_all.json','r') as f:
    normal = json.load(f)

In [6]:
clue0529.keys()

dict_keys(['data', 'creator', 'version'])

In [8]:
len(clue0529['data'])

34500

In [7]:
clue0529['data'][0]

{'source': 4,
 'paragraphs': [{'qas': [{'classtype': 'work_why',
     'id': 'm6_409391-1',
     'answers': [{'text': '수도 동파', 'answer_start': 144}],
     'question': '재개발단지에 남아있는 주민들이 불편을 겪는 이유는 뭐야?',
     'clue': [{'clue_start': 173,
       'clue_text': '미처 이사를 가지 못한 거주민들은 새는 물을 대야로 바치거나 습기가 가득 찬 마루 바닥을 온풍기로 말리며 힘겨운 겨울을 나고 있다.'}]},
    {'classtype': 'work_why',
     'id': 'm6_409391-2',
     'answers': [{'text': '수도 동파', 'answer_start': 144}],
     'question': '재개발단지에 남아있는 주민들이 불편을 겪는 이유는 무엇이야?',
     'clue': [{'clue_start': 173,
       'clue_text': '미처 이사를 가지 못한 거주민들은 새는 물을 대야로 바치거나 습기가 가득 찬 마루 바닥을 온풍기로 말리며 힘겨운 겨울을 나고 있다.'}]},
    {'classtype': 'work_what',
     'id': 'm6_409392-1',
     'answers': [{'text': '현관등이 있는 천정에서 물이 새면서 전기 누전으로 연기가 나고, 고드름을 제거하다 실수로 베란다 창문마저 깨졌다. ',
       'answer_start': 290}],
     'question': '정모씨(79)는 수도 동파로 인해 어떤 피해를 입었어?',
     'clue': [{'clue_start': 354,
       'clue_text': '정씨는 &quot;이 엄동설한에 어디로 나가야 하는 지 막막하다&quot;며 &quot;여기 저기 물이 새 집 안에서도 신발을 신고 다녀

In [9]:
noanswer.keys()

dict_keys(['creator', 'version', 'data'])

In [12]:
len(noanswer['data'])

20030

In [11]:
noanswer['data'][0]

{'source': 2,
 'title': '2654.txt',
 'paragraphs': [{'qas': [{'classtype': 'work_who',
     'id': 'm3_253409-1',
     'question': '제너시스비비큐의 대표이사는 누구야?'},
    {'classtype': 'work_when',
     'id': 'm3_253410-1',
     'question': '제너시스 비비큐의 창립일은 언제야?'},
    {'classtype': 'work_where',
     'id': 'm3_253411-1',
     'question': '공정거래위원회는 어디에 사무실이 위치해있어?'},
    {'classtype': 'work_how',
     'id': 'm3_253412-1',
     'question': '공정거래위원회가 제너시스비비큐에게 내린 시정명령은 어떤 내용이야?'},
    {'classtype': 'work_how',
     'id': 'm3_253413-1',
     'question': '다른 회사는 상품권 발행비용 명목으로 수수료를 얼마씩 받고있어?'},
    {'classtype': 'work_why',
     'id': 'm3_253414-1',
     'question': '왜 제너시스비비큐는 상품권 수령을 강요하면서까지 상품권 수수료를 챙기려고 했어?'}],
   'context': '공정거래위원회는 상품권 발행비용 명목으로 고율의 발행수수료를 가맹점주에게 부담토록 한 제너시스비비큐(BBQ)에게 시정명령을 내렸다. 공정위에 따르면 제너시스비비큐는 2011년 9월부터 지난해 7월까지 가맹점이 소비자로부터 받은 상품권을 정산하면서 액면가의 10%를 발행수수료로 공제했다. 제너시스비비큐는 또 높은 수수료 부담을 피하기 위해 고객들로부터 상품권을 받지 않은 가맹점에는 가맹계약을 해지할 수 있다는 증명을 발송해 상품권 수령을 강요하기도 했다. 이를 통해 본사가 챙긴 상품권 수수료는 2

In [13]:
normal.keys()

dict_keys(['creator', 'version', 'data'])

In [14]:
len(normal['data'])

47314

In [15]:
normal['data'][0]

{'title': '15657',
 'paragraphs': [{'context': "한국청소년단체협의회와 여성가족부는 22일부터 28일까지 서울과 충북 괴산에서 '국제청소년포럼'을 연다고 21일 밝혔다. 한국 미국 캐나다 호주 등 전 세계 32개국 75여명의 대학생, 청소년들이 모여 전 세계적 현안문제에 대한 대안과 해결책을 모색하는 자리다. 이번 포럼의 주제는 '청소년과 뉴미디어'다. 스마트폰 SNS 태블릿PC 등 새로운 커뮤니케이션 매체인 '뉴미디어'에 대한 성찰과 문제점에 대해 토론한다. 기조강연을 시작으로 국가별 주제관련 사례발표, 그룹 토론 및 전체총회, '청소년선언문' 작성 및 채택 등 다양한 프로그램을 운영한다. 개회식은 22일 서울 방화동에 있는 국제청소년센터 국제회의장에서 한다. 전 세계 32개국 대학생ㆍ청소년 참가자와 전국의 청소년기관단체장과 청소년지도자 여성가족부 주한외교사절 등 100여명이 참석할 예정이다. 23일에는 유엔미래포럼 박영숙 대표가 '뉴미디어의 균형 있는 발전을 위한 청소년의 역할'에 대해 기조강연을 한다. 뉴미디어의 올바른 활용방안과 청소년문화의 형성에 대해 설명할 계획이다. 27일 폐회식에서는 '청소년선언문'을 채택한다. 선언문에는 전 세계적으로 뉴미디어의 바람직한 발전을 촉구하며 각국 청년들이 함께 실천할 수 있는 내용 등이 담길 예정이다. 한국청소년단체협의회는 포럼이 끝난 뒤 UN 등 국제기구와 참가자 각국 정부 등 국제사회에 선언문을 전달할 예정이다.",
   'qas': [{'question': "서울과 충북 괴산에서 '국제청소년포럼'을 여는 곳은?",
     'answers': [{'answer_start': 0, 'text': '한국청소년단체협의회와 여성가족부'}],
     'id': 'c1_57059-1',
     'classtype': 'work_who'},
    {'question': "'국제 청소년포럼'이 열리는 때는?",
     'answers': [{'answer_start': 19, 'te